th[0] Fundos
th[1] Setor
th[2] Preço Atual
th[3] Liquidez Diária
th[4] P/VP
th[6] Dividend Yield
th[17] Patrimônio Líquido
th[24] Quant. Ativos
th[25] Volatilidade
th[26] Num. Cotistas

In [157]:
#Importação das bibliotecas
import requests 
import json
import pandas as pd
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [158]:
#Construção do header - Fonte:https://www.youtube.com/watch?v=o7CYPXmnUMs
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "cookie": "suno_checkout_userid=ad336406-522a-42e8-ac91-349945b25c8a; _gcl_au=1.1.2079664177.1741828992; __gtm_referrer=https%3A%2F%2Fwww.google.com%2F; _clck=1ot281v%7C2%7Cfu6%7C0%7C1898; _hjSession_1183503=eyJpZCI6IjE4YjkxMmFjLTIxNGItNGZlNC05NGFlLTE5YmMwODZmNzA5MiIsImMiOjE3NDE4Mjg5OTMyNDcsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxLCJzcCI6MH0=; _ga=GA1.1.630256352.1741828993; FPAU=1.1.2079664177.1741828992; __hstc=173632188.cd3633df4a95797d9d484304826184c0.1741828994294.1741828994294.1741828994294.1; hubspotutk=cd3633df4a95797d9d484304826184c0; __hssrc=1; hotid=eyJjaWQiOiIxNzMyNjQ5NzkxODA0MzY3MDk3MTk2ODE4MzQ3ODQwIiwiYmlkIjoiMTczMjY0OTc5MTgwNDM2NzA5NzE5NjgxODM0Nzg0MCIsInNpZCI6ImVmNTZlNzQ0MTRkYTQ5NDRiOTI3Y2I3NmZjNGEzZjRkIn0=; wisepops_visits=%5B%222025-03-13T01%3A23%3A12.327Z%22%5D; _fbp=fb.2.1741828995015.675119474247774340; messagesUtk=cb068ce4c95944d28cd71ef425d1f3ae; hs-messages-hide-welcome-message=true; FPGSID=1.1741828994.1741829763.G-VN673HEDSJ.cBnUWwf23EDte3Fu41fcVQ; cf_clearance=ovlcDHQmQQwMnCO6MYDq9tfCCCtro5PlVNXDTo5rrVM-1741829764-1.2.1.1-JHdUrEIU2rbFSDmk00ws4TyYnX0TulgvETPCyo1.wJd5Mn2YbJfIO2M6CKzbCvc_UGKblzViUaamovWVUkweH28HmjAVDFGLAgb3zsa2Fq..5l1V7BtkuGy838swD4JcySezQ_uXw7I7EHeJa6B3hL5XjvfrQap9WfeHp45S.LVyB5D7efwVNtvRNLXJu0YCXNf_uwEPsjNJ2A0UrV2_q3jFsF7Cbk72eTFPeOZ.Uq6Rox3afoauE31KJG2bcFbZZVCVkSQA7Nd3ieYh4Lbwudnd32kIPrWsWJsltSscHCcpZbvTJ2UU5hgH.KrLA3n9aYntEGGlVIIgSHFJeAcPQUmwOe6oy4xvkafcQeS2Ar4; _hjSessionUser_1183503=eyJpZCI6IjEyOTFhYmNjLTk3ZmEtNTVkNy1hMmY2LWNlYmJlODI1ZThkNSIsImNyZWF0ZWQiOjE3NDE4Mjg5OTMyNDYsImV4aXN0aW5nIjp0cnVlfQ==; wisepops=%7B%22popups%22%3A%7B%7D%2C%22sub%22%3A0%2C%22ucrn%22%3A2%2C%22cid%22%3A%2252100%22%2C%22v%22%3A5%2C%22bandit%22%3A%7B%22recos%22%3A%7B%7D%7D%7D; _clsk=1w19r3t%7C1741829776510%7C4%7C1%7Cf.clarity.ms%2Fcollect; __hssc=173632188.3.1741828994294; wisepops_visitor=%7B%22Uiy2Q6Vhk4%22%3A%22079c853a-e2a4-4608-8104-1a4d7d24a124%22%7D; wisepops_session=%7B%22arrivalOnSite%22%3A%222025-03-13T01%3A23%3A12.327Z%22%2C%22mtime%22%3A1741829776740%2C%22pageviews%22%3A3%2C%22popups%22%3A%7B%7D%2C%22bars%22%3A%7B%7D%2C%22embeds%22%3A%7B%7D%2C%22sticky%22%3A%7B%7D%2C%22countdowns%22%3A%7B%7D%2C%22src%22%3A%22https%3A%2F%2Fwww.google.com%2F%22%2C%22utm%22%3A%7B%7D%2C%22testIp%22%3Anull%7D; _ga_VN673HEDSJ=GS1.1.1741828993.1.1.1741829790.0.0.1432253131",
    "priority": "u=1, i",
    "referer": "https://www.fundsexplorer.com.br/ranking",
    "sec-ch-ua": '"Chromium";v="134", "Not:A-Brand";v="24", "Google Chrome";v="134"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "x-funds-nonce": "61495f60b533cc40ad822e054998a3190ea9bca0d94791a1da"
}

In [159]:
## Declaração da url a ser utilizada
url= 'https://www.fundsexplorer.com.br/wp-json/funds/v1/get-ranking'

In [160]:
## Realizando a requisição
response = requests.get(url, headers=headers).json() 
response = json.loads(response)
df_fiis = pd.DataFrame(response)

# Configura pandas para mostrar todas as colunas
pd.set_option('display.max_columns', None)



In [161]:
## Colocando o nome das colunas em caixa alta
df_fiis.columns = df_fiis.columns.str.upper() #Colocar as colunas em caixa alta
df_fiis.columns

Index(['POST_ID', 'TICKER', 'DIVIDENDO', 'YELD', 'MEDIA_YIELD_3M',
       'SOMA_YIELD_3M', 'MEDIA_YIELD_6M', 'SOMA_YIELD_6M', 'MEDIA_YIELD_12M',
       'SOMA_YIELD_12M', 'VARIACAO_COTACAO_MES', 'RENTABILIDADE',
       'RENTABILIDADE_MES', 'COTACAO_FECHAMENTO', 'SOMA_YIELD_ANO_CORRENTE',
       'ANO', 'VPA_YIELD', 'VPA', 'VPA_CHANGE', 'PL', 'VPA_RENT', 'VPA_RENT_M',
       'YIELD_VPA_3M_SUM', 'YIELD_VPA_3M', 'YIELD_VPA_6M_SUM', 'YIELD_VPA_6M',
       'YIELD_VPA_12M_SUM', 'YIELD_VPA_12M', 'SETOR', 'SETOR_SLUG', 'VALOR',
       'LIQUIDEZMEDIADIARIA', 'PATRIMONIO', 'PVP', 'P_VPA', 'POST_TITLE',
       'ATIVOS', 'VOLATILITY', 'NUMERO_COTISTA', 'TX_GESTAO', 'TX_ADMIN',
       'TX_PERFORMANCE'],
      dtype='object')

In [162]:
## Selecionando as colunas para utilização - Ajuste os nomes conforme necessário
colunas = ['TICKER','SETOR','COTACAO_FECHAMENTO','VALOR','LIQUIDEZMEDIADIARIA','PVP','YELD','MEDIA_YIELD_12M','SOMA_YIELD_ANO_CORRENTE','RENTABILIDADE','PL','PATRIMONIO','NUMERO_COTISTA', 'TX_GESTAO']  

In [163]:
# Criando um dataframe
df_fiis = pd.DataFrame(df_fiis, columns=colunas)
df_fiis


,TICKER,SETOR,COTACAO_FECHAMENTO,VALOR,LIQUIDEZMEDIADIARIA,PVP,YELD,MEDIA_YIELD_12M,SOMA_YIELD_ANO_CORRENTE,RENTABILIDADE,PL,PATRIMONIO,NUMERO_COTISTA,TX_GESTAO
0,AAGR11,Indefinido,96.1400,99,13691.55,0,1.2100,1.2625,5.2300,10.0613,0.0000,0,0,
1,AAZQ11,Indefinido,7.3900,7.22,717526.55,0.8316,1.2900,1.4917,6.2400,19.8465,206946884.0600,206946884.06,29817,
2,ABCP11,Shoppings,75.8000,78.24,44393.65,0.7268,0.6600,0.8692,2.7800,17.5049,507004106.3900,507004106.39,14920,
3,AFHI11,Papéis,92.7500,93,756855.8,0.9946,1.0700,1.0650,4.3100,11.4197,424592667.1100,424592667.11,42402,
4,AGRX11,Indefinido,8.0900,8,205026,0.784,1.3600,1.3467,5.6600,22.9554,183235232.6600,183235232.66,19257,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,YUFI11B,Indefinido,0.0000,0,,0,0.0000,0.0000,None,0.0000,41701052.9800,0,0,
522,ZAGH11,Serviços Financeiros Diversos,10.2100,10.4,1857.2,1.1429,0.6900,0.5608,2.7900,-17.0124,None,80888876.47,408,
523,ZAVC11,Indefinido,None,8.6,48089.25,0.8516,0.0000,1.3091,0.0000,None,None,30880981.49,195,
524,ZAVI11,Indefinido,93.8300,93.3,103575.05,0.7063,1.1200,1.1600,4.9300,0.5321,None,153693067.63,3332,


##### Tratamento de Dados

COTACAO_FECHAMENTO - R$<br>
VALOR - R$<br>
LIQUIDEZMEDIADIARIA - R$<br>
PVP - 2 casas após ','<br>
YELD - %<br>
MEDIA_YIELD_12M - %<br>
SOMA_YIELD_ANO_CORRENTE %<br>
RENTABILIDADE - %

In [164]:
## União das colunas PL e Patrimonio Liquido, respeitando os valores existentes
df_fiis['PATRIMONIO_LIQUIDO'] = (df_fiis['PL'].combine_first(df_fiis['PATRIMONIO']))  # Prioriza PL, depois PATRIMONIO
df_fiis

,TICKER,SETOR,COTACAO_FECHAMENTO,VALOR,LIQUIDEZMEDIADIARIA,PVP,YELD,MEDIA_YIELD_12M,SOMA_YIELD_ANO_CORRENTE,RENTABILIDADE,PL,PATRIMONIO,NUMERO_COTISTA,TX_GESTAO,PATRIMONIO_LIQUIDO
0,AAGR11,Indefinido,96.1400,99,13691.55,0,1.2100,1.2625,5.2300,10.0613,0.0000,0,0,,0.0000
1,AAZQ11,Indefinido,7.3900,7.22,717526.55,0.8316,1.2900,1.4917,6.2400,19.8465,206946884.0600,206946884.06,29817,,206946884.0600
2,ABCP11,Shoppings,75.8000,78.24,44393.65,0.7268,0.6600,0.8692,2.7800,17.5049,507004106.3900,507004106.39,14920,,507004106.3900
3,AFHI11,Papéis,92.7500,93,756855.8,0.9946,1.0700,1.0650,4.3100,11.4197,424592667.1100,424592667.11,42402,,424592667.1100
4,AGRX11,Indefinido,8.0900,8,205026,0.784,1.3600,1.3467,5.6600,22.9554,183235232.6600,183235232.66,19257,,183235232.6600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,YUFI11B,Indefinido,0.0000,0,,0,0.0000,0.0000,None,0.0000,41701052.9800,0,0,,41701052.9800
522,ZAGH11,Serviços Financeiros Diversos,10.2100,10.4,1857.2,1.1429,0.6900,0.5608,2.7900,-17.0124,None,80888876.47,408,,80888876.47
523,ZAVC11,Indefinido,None,8.6,48089.25,0.8516,0.0000,1.3091,0.0000,None,None,30880981.49,195,,30880981.49
524,ZAVI11,Indefinido,93.8300,93.3,103575.05,0.7063,1.1200,1.1600,4.9300,0.5321,None,153693067.63,3332,,153693067.63


In [165]:
## Transformando as colunas None em "---"(Colunas que os dados não foram imputados)
df_fiis = df_fiis.fillna("---")
df_fiis = df_fiis.drop(["PL","PATRIMONIO"], axis=1)


In [166]:
##Corrigir valores da coluna setor
df_fiis['SETOR'] = df_fiis['SETOR'].replace({'Indefinido': 'Outros', 'Pagels': 'Papéis'})
df_fiis

,TICKER,SETOR,COTACAO_FECHAMENTO,VALOR,LIQUIDEZMEDIADIARIA,PVP,YELD,MEDIA_YIELD_12M,SOMA_YIELD_ANO_CORRENTE,RENTABILIDADE,NUMERO_COTISTA,TX_GESTAO,PATRIMONIO_LIQUIDO
0,AAGR11,Outros,96.1400,99,13691.55,0,1.2100,1.2625,5.2300,10.0613,0,,0.0000
1,AAZQ11,Outros,7.3900,7.22,717526.55,0.8316,1.2900,1.4917,6.2400,19.8465,29817,,206946884.0600
2,ABCP11,Shoppings,75.8000,78.24,44393.65,0.7268,0.6600,0.8692,2.7800,17.5049,14920,,507004106.3900
3,AFHI11,Papéis,92.7500,93,756855.8,0.9946,1.0700,1.0650,4.3100,11.4197,42402,,424592667.1100
4,AGRX11,Outros,8.0900,8,205026,0.784,1.3600,1.3467,5.6600,22.9554,19257,,183235232.6600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,YUFI11B,Outros,0.0000,0,,0,0.0000,0.0000,---,0.0000,0,,41701052.9800
522,ZAGH11,Serviços Financeiros Diversos,10.2100,10.4,1857.2,1.1429,0.6900,0.5608,2.7900,-17.0124,408,,80888876.47
523,ZAVC11,Outros,---,8.6,48089.25,0.8516,0.0000,1.3091,0.0000,---,195,,30880981.49
524,ZAVI11,Outros,93.8300,93.3,103575.05,0.7063,1.1200,1.1600,4.9300,0.5321,3332,,153693067.63


In [167]:
print("Colunas existentes:", df_fiis.columns.tolist())

Colunas existentes: ['TICKER', 'SETOR', 'COTACAO_FECHAMENTO', 'VALOR', 'LIQUIDEZMEDIADIARIA', 'PVP', 'YELD', 'MEDIA_YIELD_12M', 'SOMA_YIELD_ANO_CORRENTE', 'RENTABILIDADE', 'NUMERO_COTISTA', 'TX_GESTAO', 'PATRIMONIO_LIQUIDO']
